## Load modules and read in data

First, we will import all the modules needed for this analysis.

In [1]:
import pandas as pd
from pyspi.calculator import Calculator
import numpy as np
from copy import deepcopy
import glob
import os
import random

# Define data directory
data_dir = "data/numpy_arrays/"

## Getting started with pyspi

Let's go through some of the basics with `pyspi` functionality to warm up. The core object in `pyspi` is a `Calculator`, which stores your input multivariate time series (MTS) data, configuration parameters, and ultimately the computed metrics -- called "statistics of pairwise interactions", or "SPIs" for short.

By default, when you initialise a `Calculator` object, it loads in all 283 available SPIs, which range from basic statistics like the Pearson correlation coefficient to frequency-based metrics like the spectral Granger causality. In this project, we will focus on just one SPI: directed information with a Gaussian density estimator, which is denoted as `di_gaussian` within `pyspi`. To save time and computational power, instead of computing all 283 SPIs, we can load our `Calculator` object with a `YAML` configuration file that only specifies the `di_gaussian` metric: `DI_Gaussian_config.yaml`, contained in this repo.

In [2]:
# Define configuration file
di_gaussian_config_file = "DI_Gaussian_config.yaml"

# Initialise a Calculator object with this configuration file
calc = Calculator(configfile=di_gaussian_config_file)

Loading configuration file: DI_Gaussian_config.yaml
*** Importing module .statistics.infotheory
[0] Adding SPI .statistics.infotheory.DirectedInfo(x,y,{'estimator': 'gaussian'})...
Succesfully initialised SPI with identifier "di_gaussian" and labels ['unsigned', 'infotheory', 'temporal', 'directed', 'linear']
Number of SPIs: 1


Now that we have created our Calculator object, we can load in an MTS matrix in the form of a 2D `NumPy` array. For now, we can simulate some random data to demonstrate how this process works:

In [3]:
# Set a seed for reproducibility
random.seed(42)

# Create a random dataset consisting of 3 processes and 100 time points
example_data = np.random.randn(3,100)

# Load example_data into the Calculator object
calc.load_dataset(example_data)


To run compute our SPI of interest, `di_gaussian`, all we need to do is call the `compute()` method on our `calc` object:

In [4]:
# Compute the SPI (di_gaussian) defined in the Calculator object
calc.compute()

# Our results are stored in the calc.table object:
calc.table

Processing [None: di_gaussian]: 100%|██████████| 1/1 [00:00<00:00, 15.84it/s]


spi     di_gaussian                    
process      proc-0    proc-1    proc-2
proc-0          NaN  0.093598  0.102576
proc-1     0.100323       NaN  0.103039
proc-2     0.102620  0.121182       NaN

In this `DataFrame` we can see that the SPI is `di_gaussian` and we now have a 3x3 `NumPy` array containing the SPI values for each pair of processes (which in our case, will represent brain regions). There are two key things to note:
1. The diagonal of the matrix contains all `NaN`, because these represent self-connections and therefore are not valid for pairwise interaction analysis; and 
2. The matrix is not symmetrical -- meaning that the statistics are not the same going from proc-0 to proc-1 as the other way around. This is because `di_gaussian` is a **directed** statistic, meaning that it is sensitive to the flow of information from one time series to the other.

If we want to loop over a few `DataFrames` and compute the `di_gaussian` for each pair of time-series per `DataFrame`, here's how we could do that with a for loop:

In [5]:
di_gaussian_list = []

# For five loops, create a random dataset and compute the DI
for i in range(5):
    # Create a random dataset consisting of 3 processes and 100 time points
    example_data = np.random.randn(3,100)

    # Create a Calculator object
    calc = Calculator(configfile=di_gaussian_config_file)
    
    # Load example_data into the Calculator object
    calc.load_dataset(example_data)
    
    # Compute the SPI (di_gaussian) defined in the Calculator object
    calc.compute()
    
    # Extract directed information results as the calc.table object
    di_gaussian_loop = calc.table

    # Save the loop number as a column
    di_gaussian_loop['loop'] = i
    
    # Append the results to the di_gaussian_list
    di_gaussian_list.append(di_gaussian_loop)

# Concatenate the list of dataframes into a single dataframe
di_gaussian = pd.concat(di_gaussian_list, axis=0)

Processing [None: di_gaussian]: 100%|██████████| 1/1 [00:00<00:00, 53.80it/s]

Loading configuration file: DI_Gaussian_config.yaml
*** Importing module .statistics.infotheory
[0] Adding SPI .statistics.infotheory.DirectedInfo(x,y,{'estimator': 'gaussian'})...
Succesfully initialised SPI with identifier "di_gaussian" and labels ['unsigned', 'infotheory', 'temporal', 'directed', 'linear']
Number of SPIs: 1
Loading configuration file: DI_Gaussian_config.yaml
*** Importing module .statistics.infotheory
[0] Adding SPI .statistics.infotheory.DirectedInfo(x,y,{'estimator': 'gaussian'})...
Succesfully initialised SPI with identifier "di_gaussian" and labels ['unsigned', 'infotheory', 'temporal', 'directed', 'linear']
Number of SPIs: 1
Loading configuration file: DI_Gaussian_config.yaml
*** Importing module .statistics.infotheory
[0] Adding SPI .statistics.infotheory.DirectedInfo(x,y,{'estimator': 'gaussian'})...
Succesfully initialised SPI with identifier "di_gaussian" and labels ['unsigned', 'infotheory', 'temporal', 'directed', 'linear']
Number of SPIs: 1
Loading confi

In [6]:
# Print the results
print(di_gaussian)

spi     di_gaussian                     loop
process      proc-0    proc-1    proc-2     
proc-0          NaN  0.002363  0.111854    0
proc-1     0.049948       NaN  0.092539    0
proc-2     0.043522  0.117353       NaN    0
proc-0          NaN  0.038890  0.111603    1
proc-1     0.087555       NaN  0.061707    1
proc-2     0.137940  0.064682       NaN    1
proc-0          NaN  0.156348  0.184962    2
proc-1     0.078915       NaN  0.100849    2
proc-2     0.072611  0.085196       NaN    2
proc-0          NaN  0.162005  0.135713    3
proc-1     0.135675       NaN  0.075036    3
proc-2     0.108033  0.116255       NaN    3
proc-0          NaN  0.096205  0.267372    4
proc-1     0.081094       NaN  0.120448    4
proc-2     0.184391  0.154047       NaN    4


The above code chunk collects the `di_gaussian` matrices for each loop into one `DataFrame` for downstream analysis -- we'll use this same approach to iterate over all brain regions per participant in the Human Connectome Project dataset, computing the `di_gaussian` between the left and right hemispheres. 

Instead of re-initializing a `Calculator` object for every single iteration, we can instead create one base `Calculator` object and then make a copy of it for each iteration using the `deepcopy` function from the `copy` module:

In [7]:
di_gaussian_list = []

# Create a base Calculator object
basecalc = Calculator(configfile="DI_Gaussian_config.yaml")

# For five loops, create a random dataset and compute the DI
for i in range(5):
    # Create a random dataset consisting of 3 processes and 100 time points
    example_data = np.random.randn(3,100)

    # Create a deepcopy of the base Calculator object
    calc = deepcopy(basecalc)
    
    # Load example_data into the Calculator object
    calc.load_dataset(example_data)
    
    # Compute the SPI (di_gaussian) defined in the Calculator object
    calc.compute()
    
    # Extract directed information results as the calc.table object
    di_gaussian_loop = calc.table

    # Save the loop number as a column
    di_gaussian_loop['loop'] = i
    
    # Append the results to the di_gaussian_list
    di_gaussian_list.append(di_gaussian_loop)

# Concatenate the list of dataframes into a single dataframe
di_gaussian = pd.concat(di_gaussian_list, axis=0)

Processing [None: di_gaussian]: 100%|██████████| 1/1 [00:00<00:00, 56.42it/s]

Loading configuration file: DI_Gaussian_config.yaml
*** Importing module .statistics.infotheory
[0] Adding SPI .statistics.infotheory.DirectedInfo(x,y,{'estimator': 'gaussian'})...
Succesfully initialised SPI with identifier "di_gaussian" and labels ['unsigned', 'infotheory', 'temporal', 'directed', 'linear']
Number of SPIs: 1


In [8]:
# Print the results
print(di_gaussian)

spi     di_gaussian                     loop
process      proc-0    proc-1    proc-2     
proc-0          NaN  0.085199  0.095575    0
proc-1     0.018247       NaN  0.103148    0
proc-2     0.149342  0.031945       NaN    0
proc-0          NaN  0.074876  0.023074    1
proc-1     0.026809       NaN  0.043553    1
proc-2     0.132912  0.137522       NaN    1
proc-0          NaN  0.197936  0.084086    2
proc-1     0.127262       NaN  0.243696    2
proc-2     0.128574  0.165881       NaN    2
proc-0          NaN  0.121764  0.101879    3
proc-1     0.061805       NaN  0.072775    3
proc-2     0.121983  0.131444       NaN    3
proc-0          NaN  0.037242  0.200777    4
proc-1     0.091780       NaN  0.183810    4
proc-2     0.093570  0.080710       NaN    4


## Working with real neuroimaging data from the Human Connectome Project

For our case study, we will be working with high-quality resting-state functional magnetic resonance imaging (fMRI) data from 100 participants in the Human Connectome Project (HCP100). 

Voxels in the cortex (gray matter on the brain's surface) can be divided into brain regions in many different ways; in this dataset, each participant's cortex was segmented into 68 regions, comprising 34 unique regions in each hemisphere (right and left). 
This atlas -- the Desikan-Killiany atlas -- is shown at the top left of the following figure, originally published in [Fürtjes et al. *Cortex* (2023)](https://doi.org/10.1016/j.cortex.2022.11.001):  

<img src="https://ars.els-cdn.com/content/image/1-s2.0-S0010945222003008-gr1.jpg" width="50%" height="50%" />

In fMRI data, what we measure is blood oxygen-level dependent (BOLD) signaling that reflects the ratio of oxygenated versus deoxygenated hemoglobin in the blood. This is a proxy readout for neural activity, since increased activity in a brain region will prompt increased blood flow to that region through a pathway called neurovascular coupling.
For each voxel, the BOLD signal is measured over 1,200 time points; we then aggregate across all the voxels in a given region to get the average BOLD time series for that region.
This means that for each subject in the HCP100 dataset, we have a 68 x 1200 matrix for all brain regions by time points.

This fMRI data was already preprocessed as part of [Fallon et al. *Network Neuroscience* (2020)](https://pubmed.ncbi.nlm.nih.gov/33615091/). The code for this paper can be found at [https://github.com/DynamicsAndNeuralSystems/humanStructureFunction](https://github.com/DynamicsAndNeuralSystems/humanStructureFunction) and the data can be downloaded from [Zenodo](https://zenodo.org/record/4643074).

For our purposes, we have already parsed each subject's Matlab `.mat` files into individual `NumPy` arrays that are stored in `.npy` files for easy loading into Python.
These are included in the [data/numpy_arrays/](https://github.com/DynamicsAndNeuralSystems/Directed_information_fMRI/tree/main/data/numpy_arrays) folder in this repo.

Let's load in one of these files with the `np.load()` function (where `np` is an alias for `numpy`) and examine the data structure to know what we're working with:

In [9]:
import numpy as np

# Load in the numpy array for subject 100307
subject = "100307"
subject_data = np.load(f"data/numpy_arrays/{subject}.npy")

# Print the first 5 rows and first 20 columns of subject_data
print(subject_data[:5, :20])

[[-1.60778374 -0.98263907 -0.68103114  0.43221171 -0.54265908  0.19205564
   0.73322423  0.73347631  1.04091909 -0.26856168  1.09567074  1.64613661
   1.28347511 -0.06633166  0.58277635 -1.99615489 -0.4640819  -1.27861954
   1.09482733  1.09997448]
 [-0.166792   -1.9242528  -1.50736902 -0.60595182 -0.3713573   1.45826845
  -0.33929535  0.53375432  0.55757112  0.54106949  0.94932919  1.48659255
   0.66942609  1.40101728  2.54179148  0.61465474  0.87365191  1.03290417
   0.15311191  2.00881693]
 [-0.81218222 -0.36716754  0.03075874  2.13221635  0.06484777  0.41182043
   0.4203798   0.45791618  0.6308759   0.26260445 -0.97917152 -1.16845825
  -0.31068965  1.11679565  0.3130997   1.54490938  0.73089731 -0.00552274
  -0.3219353  -1.35928571]
 [ 0.35714679 -0.88998292 -1.25442004 -2.00446651 -1.24282621 -1.77447153
  -1.18750537 -1.29986359 -0.40973432 -0.25304476  0.03764887 -0.19349954
   0.14567891 -1.22013181 -1.31565385 -0.38236477 -0.19735502  1.32893186
   0.69123047 -0.64406845]
 [ 0

Here, we've printed out the first 5 rows (corresponding to the first 5 brain regions) and the first 20 columns (corresponding to the first 20 time points) of the 68-by-1200 array.

Note that the array doesn't store the name of any brain region, so we'll need to link this information later.
We provide a lookup table to go between row index and region name in [Brain_Region_info.csv](https://github.com/DynamicsAndNeuralSystems/Directed_information_fMRI/blob/main/Brain_Region_info.csv).

We can load this lookup table in here:

In [10]:
brain_region_lookup = pd.read_csv("Brain_Region_info.csv", index_col=False).reset_index(drop=True)

brain_region_lookup

,Region_Index,Brain_Region,ggseg,Hemisphere,Base_Region
0,0,ctx-lh-bankssts,ctx-lh-bankssts,Left,bankssts
1,1,ctx-lh-caudalanteriorcingulate,ctx-lh-caudalanteriorcingulate,Left,caudalanteriorcingulate
2,2,ctx-lh-caudalmiddlefrontal,ctx-lh-caudalmiddlefrontal,Left,caudalmiddlefrontal
3,3,ctx-lh-cuneus,ctx-lh-cuneus,Left,cuneus
4,4,ctx-lh-entorhinal,ctx-lh-entorhinal,Left,entorhinal
...,...,...,...,...,...
63,63,ctx-rh-supramarginal,ctx-rh-supramarginal,Right,supramarginal
64,64,ctx-rh-frontalpole,ctx-rh-frontalpole,Right,frontalpole
65,65,ctx-rh-temporalpole,ctx-rh-temporalpole,Right,temporalpole
66,66,ctx-rh-transversetemporal,ctx-rh-transversetemporal,Right,transversetemporal


As this lookup table shows, the first column corresponds to the left banks of the superior temporal sulcus (bankssts), which is designated with the brain region name `ctx-lh-bankssts`. The `Base_Region` column denotes the unique brain region names, each of which has a left and right hemisphere component. We can get this into a list that we'll iterate over for each subject later on:

In [11]:
base_regions = list(set(brain_region_lookup.Base_Region.tolist()))

# Print the names of our 34 cortical brain regions
print(base_regions)


['inferiorparietal', 'lateraloccipital', 'middletemporal', 'fusiform', 'entorhinal', 'inferiortemporal', 'parstriangularis', 'parahippocampal', 'superiortemporal', 'paracentral', 'supramarginal', 'lingual', 'superiorfrontal', 'transversetemporal', 'caudalanteriorcingulate', 'frontalpole', 'rostralmiddlefrontal', 'lateralorbitofrontal', 'temporalpole', 'isthmuscingulate', 'pericalcarine', 'parsorbitalis', 'precuneus', 'bankssts', 'cuneus', 'posteriorcingulate', 'superiorparietal', 'insula', 'caudalmiddlefrontal', 'precentral', 'parsopercularis', 'rostralanteriorcingulate', 'medialorbitofrontal', 'postcentral']


Let's try computing `di_gaussian` for the left and right `transversetemporal` cortex for subject `100307`.
To achieve this, we'll need to find the row number (index) for the left and right `transversetemporal` regions in `brain_region_lookup` and then subset the 68x1200 array to just these two rows, yielding a 2x1200 array as the input dataset to a new `Calculator` object.

In [12]:
# Find left and right transversetemporal indices
left_index = brain_region_lookup.query("Base_Region == 'transversetemporal' & Hemisphere == 'Left'").Region_Index.tolist()[0]
right_index = brain_region_lookup.query("Base_Region == 'transversetemporal' & Hemisphere == 'Right'").Region_Index.tolist()[0]
    
print("Left index: ", left_index)
print("Right index: ", right_index)

Left index:  32
Right index:  66


In [13]:
# Subset the subject_data numpy array to just left_index and right_index
left_TS = subject_data[left_index,:].reshape(1, -1)
right_TS = subject_data[right_index,:].reshape(1, -1)

# Combine left_TS and right_TS into a 2 by 1200 numpy array
bilateral_arr_to_compute = np.concatenate((left_TS, right_TS), axis=0)

# Print the dimensions of the bilateral_arr_to_compute array
print(bilateral_arr_to_compute.shape)

(2, 1200)


This new 2x1200 array contains the left hemisphere in the first row and right hemisphere in the second row, which is how we will format all brain regions across all subjects for consistency.
We can then compute the `di_gaussian` SPI between the left and right hemispheres for this subject as we did with the randomly generated `NumPy` arrays above:

In [14]:
calc = Calculator(configfile="DI_Gaussian_config.yaml")
calc.load_dataset(bilateral_arr_to_compute)
calc.compute()

# Print the results table
calc.table

Processing [None: di_gaussian]: 100%|██████████| 1/1 [00:00<00:00,  9.76it/s]

Loading configuration file: DI_Gaussian_config.yaml
*** Importing module .statistics.infotheory
[0] Adding SPI .statistics.infotheory.DirectedInfo(x,y,{'estimator': 'gaussian'})...
Succesfully initialised SPI with identifier "di_gaussian" and labels ['unsigned', 'infotheory', 'temporal', 'directed', 'linear']
Number of SPIs: 1


spi     di_gaussian          
process      proc-0    proc-1
proc-0          NaN  0.107727
proc-1     0.114898       NaN

Here, `proc-0` refers to the left hemisphere and `proc-1` refers to the right hemisphere.
This format isn't the most conducive to combining across subjects and brain regions, so we can reshape it to be in a long format:

In [15]:
# Extract directed information results as the calc.table object
ROI_DI_data = deepcopy(calc.table)

# Flatten the MultiIndex columns to just the process IDs
ROI_DI_data.columns = ROI_DI_data.columns.get_level_values(1)

# Pivot and clean up ROI directed information data
ROI_DI_data = (ROI_DI_data
                .reset_index(level=0) # Convert index to column
                .rename(columns={"index": "Hemi_from"}) # Rename index as first hemisphere
                .melt(id_vars="Hemi_from") # Pivot data from wide to long
                .rename(columns={"process": "Hemi_to"}) # Rename hemisphere receiving the connection
                .query("Hemi_from != Hemi_to") # Remove self-connections
                .assign(Sample_ID = "100307",
                        Brain_Region = "transversetemporal")
                .assign(Hemi_from=lambda x: x['Hemi_from'].replace({'proc-0': 'left', 'proc-1': 'right'}),
                        Hemi_to=lambda x: x['Hemi_to'].replace({'proc-0': 'left', 'proc-1': 'right'}))
                )

# Print the resulting dataframe
ROI_DI_data

,Hemi_from,Hemi_to,value,Sample_ID,Brain_Region
1,right,left,0.114898,100307,transversetemporal
2,left,right,0.107727,100307,transversetemporal


## Create a helper function for directed information computation

Next, we'll define a helper function to compute directed information with a Gaussian density estimator for a given pair of brain regions. 
This function will read in the name of a brain region (e.g., `transversetemporal`, as above), the brain region lookup table we loaded above, the given subject's ID and region-wise time-series data, and a `basecalc` object.
It will repeat the same steps as above and output the resulting reshaped `ROI_DI_data` dataframe.


In [16]:
# Helper function to read in a base region (e.g., bankssts) and compute
# Directed information from left to right and from right to left
def compute_DI_for_region(base_region, brain_region_lookup, subject_ID, subject_data, basecalc):
    left_index = brain_region_lookup.query("Base_Region == @base_region & Hemisphere == 'Left'").Region_Index.tolist()[0]
    right_index = brain_region_lookup.query("Base_Region == @base_region & Hemisphere == 'Right'").Region_Index.tolist()[0]
    
    # Subset the subject_data numpy array to just left_index and right_index
    left_TS = subject_data[left_index,:].reshape(1, -1)
    right_TS = subject_data[right_index,:].reshape(1, -1)
    
    # Combine left_TS and right_TS into a 2 by 1200 numpy array
    bilateral_arr_to_compute = np.concatenate((left_TS, right_TS), axis=0)
    
    # Compute directed information for these two time-series
    calc = deepcopy(basecalc)
    calc.load_dataset(bilateral_arr_to_compute)
    calc.compute()
    
    # Extract directed information results as the calc.table object
    ROI_DI_data = calc.table 
    
    # Flatten the MultiIndex columns to just the process IDs
    ROI_DI_data.columns = ROI_DI_data.columns.get_level_values(1)
    
    # Pivot and clean up ROI directed information data
    ROI_DI_data = (ROI_DI_data
                   .reset_index(level=0) # Convert index to column
                   .rename(columns={"index": "Hemi_from"}) # Rename index as first hemisphere
                   .melt(id_vars="Hemi_from") # Pivot data from wide to long
                   .rename(columns={"process": "Hemi_to"}) # Rename hemisphere receiving the connection
                   .query("Hemi_from != Hemi_to") # Remove self-connections
                   .assign(Sample_ID = subject_ID,
                           Brain_Region = base_region)
                    .assign(Hemi_from=lambda x: x['Hemi_from'].replace({'proc-0': 'left', 'proc-1': 'right'}),
                            Hemi_to=lambda x: x['Hemi_to'].replace({'proc-0': 'left', 'proc-1': 'right'}))
                   )
    
    # Return the final dataframe for this region
    return ROI_DI_data

## Compute DI across all regions and all participants

The last step is to iterate over each of the HCP100 subjects and then within each subject, iterate over all 34 brain regions to compute `di_gaussian` between the left and right hemispheres.

In [17]:
# Define a base Calculator object
basecalc = Calculator(configfile="DI_Gaussian_config.yaml")

# Initialise a list for all HCP100 directed information
HCP100_DI_list = []

# Iterate over each of the 100 
for subject_filepath in glob.glob(f'{data_dir}/*.npy'):
    
    # Subset to basename for file
    subject_npy = os.path.basename(subject_filepath)
    
    # Get sample ID substring
    subject_ID = subject_npy.replace(".npy", "")
    
    # Load in subject's ROI time series from numpy binary file
    subject_data = np.load(data_dir + subject_npy)
    
    # Initialise a list to store the directed information for each region between left and right hemispheres
    subject_DI_list = []
    
    # Apply compute_DI_for_region to each base region
    for base_region in base_regions:
        ROI_DI_data = compute_DI_for_region(base_region = base_region,
                                            brain_region_lookup = brain_region_lookup,
                                            subject_ID = subject_ID,
                                            subject_data = subject_data,
                                            basecalc = basecalc)
        subject_DI_list.append(ROI_DI_data)
    
    # Concatenate dataframes across regions
    subject_DI = pd.concat(subject_DI_list, axis=0)
    
    HCP100_DI_list.append(subject_DI)

# Combine all the dataframes in the HC100_DI_list into a single dataframe
HCP100_DI = (pd.concat(HCP100_DI_list, axis=0)
             .rename(columns={"value": "di_gaussian"})
             )

Processing [None: di_gaussian]:   0%|          | 0/1 [00:00<?, ?it/s]

Loading configuration file: DI_Gaussian_config.yaml
*** Importing module .statistics.infotheory
[0] Adding SPI .statistics.infotheory.DirectedInfo(x,y,{'estimator': 'gaussian'})...
Succesfully initialised SPI with identifier "di_gaussian" and labels ['unsigned', 'infotheory', 'temporal', 'directed', 'linear']
Number of SPIs: 1


Processing [None: di_gaussian]: 100%|██████████| 1/1 [00:00<00:00, 11.06it/s]


In [18]:
# Print the head of this dataframe
HCP100_DI.head()

,Hemi_from,Hemi_to,di_gaussian,Sample_ID,Brain_Region
1,right,left,0.842470,298051,inferiorparietal
2,left,right,0.953353,298051,inferiorparietal
1,right,left,1.940216,298051,lateraloccipital
2,left,right,1.744031,298051,lateraloccipital
1,right,left,0.514164,298051,middletemporal


We can save the results to a CSV file for downstream analysis:

In [ ]:
HCP100_DI.to_csv("data/HCP100_Directed_Information.csv", index = False)

Once you reach this point and have the above CSV file in your data directory, email Annie (abry4213@uni.sydney.edu.au) and we will go forward from here :)